<a href="https://colab.research.google.com/github/aura-byte/machine-learning/blob/main/Machine_Learning_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
!unzip gdrive/MyDrive/Project-Data/test1.zip
!unzip gdrive/MyDrive/Project-Data/train.zip
drive.flush_and_unmount()

In [6]:
!zip -r model.zip saved_model
from google.colab import files
files.download("model.zip")

  adding: saved_model/ (stored 0%)
  adding: saved_model/my_model/ (stored 0%)
  adding: saved_model/my_model/saved_model.pb (deflated 87%)
  adding: saved_model/my_model/assets/ (stored 0%)
  adding: saved_model/my_model/keras_metadata.pb (deflated 88%)
  adding: saved_model/my_model/variables/ (stored 0%)
  adding: saved_model/my_model/variables/variables.index (deflated 61%)
  adding: saved_model/my_model/variables/variables.data-00000-of-00001 (deflated 26%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# create directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
    # create label subdirectories
    labeldirs = ['dogs/', 'cats/']
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        makedirs(newdir, exist_ok=True)
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'train/'
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    if file.startswith('cat'):
        dst = dataset_home + dst_dir + 'cats/'  + file
        copyfile(src, dst)
    elif file.startswith('dog'):
        dst = dataset_home + dst_dir + 'dogs/'  + file
        copyfile(src, dst)

In [4]:
import tensorflow
import sys
from matplotlib import pyplot 
from tensorflow import keras
from keras import layers
from keras import Sequential


DATASET_LOCATION = "dataset_dogs_vs_cats/"
CATS_OUTPUT = 0.0
DOGS_OUTPUT = 1.0

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()


def create_model():
    """Function will create a model and will return it."""
    model = Sequential([
        layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
        layers.Dense(1, activation='sigmoid')
        ])
        
    opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def main():
    model = create_model()
    # create data generator - scaling the pixels between 0 and 1
    datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory(DATASET_LOCATION + 'train/', class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = datagen.flow_from_directory(DATASET_LOCATION + 'test/', class_mode='binary', batch_size=64, target_size=(200, 200))
    #Save the weights using the `checkpoint_path` format
    #fit the model:
    history = model.fit(
        train_it,
        steps_per_epoch=len(train_it),
        validation_data=test_it,
        validation_steps=len(test_it), 
        epochs=20)
    # evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))
    summarize_diagnostics(history)
    model.save('saved_model/my_model')


if __name__ == "__main__":
    main()

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.
Epoch 1/20
293/293 [==============================] - 472s 2s/step - loss: 0.6901 - accuracy: 0.5664 - val_loss: 0.6586 - val_accuracy: 0.5984
Epoch 2/20
293/293 [==============================] - 451s 2s/step - loss: 0.6441 - accuracy: 0.6189 - val_loss: 0.6585 - val_accuracy: 0.6088
Epoch 3/20
293/293 [==============================] - 454s 2s/step - loss: 0.6212 - accuracy: 0.6465 - val_loss: 0.6095 - val_accuracy: 0.6576
Epoch 4/20
293/293 [==============================] - 452s 2s/step - loss: 0.6084 - accuracy: 0.6575 - val_loss: 0.6094 - val_accuracy: 0.6683
Epoch 5/20
293/293 [==============================] - 451s 2s/step - loss: 0.6008 - accuracy: 0.6659 - val_loss: 0.6015 - val_accuracy: 0.6744
Epoch 6/20
293/293 [==============================] - 449s 2s/step - loss: 0.5750 - accuracy: 0.6896 - val_loss: 0.5807 - val_accuracy: 0.6868
Epoch 7/20
293/293 [==============================] - 446